Paper: Deep Belief Network Based Machine Learning Approach for Classifying Healthcare Data
📌 Goal: Train a DBN for classification (e.g., healthcare, stroke, or any binary/multiclass task)

In [ ]:
!pip install torch torchvision pandas scikit-learn matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [ ]:
#upload data file
from google.colab import files
uploaded = files.upload()
# Load dataset
df = pd.read_csv("preprocessed_stroke_dataset.csv")

Saving preprocessed_stroke_dataset.csv to preprocessed_stroke_dataset.csv


In [ ]:
# Define a dictionary to rename columns
column_mapping = {
    "Patient ID": "patient_id",
    "Patient Name": "patient_name",
    "Age": "age",
    "Gender": "gender",
    "Hypertension": "hypertension",
    "Heart Disease": "heart_disease",
    "Marital Status": "marital_status",
    "Work Type": "work_type",
    "Residence Type": "residence_type",
    "Average Glucose Level": "glucose_level",
    "Body Mass Index (BMI)": "bmi",
    "Smoking Status": "smoking_status",
    "Alcohol Intake": "alcohol_intake",
    "Physical Activity": "physical_activity",
    "Stroke History": "personal_stroke_history",
    "Family History of Stroke": "family_history_of_stroke",
    "Dietary Habits": "dietary_habits",
    "Stress Levels": "stress_levels",
    "Blood Pressure Levels": "bp_level",
    "Cholesterol Levels": "cholesterol_level",
    "Symptoms": "symptoms",
    "Diagnosis": "diagnosis"
}

# Rename the columns
df.rename(columns=column_mapping, inplace=True)


In [ ]:
# Load dataset
df = pd.read_csv("preprocessed_stroke_dataset.csv")  # Replace with your file

# Features and labels
X = df.drop("diagnosis", axis=1)
y = df["diagnosis"]

# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)

# Feature selection - keep top 20 features (tune 'k' as needed)
selector = SelectKBest(score_func=mutual_info_classif, k=20)
X_selected = selector.fit_transform(X, y)

# Train-test split with stratification
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42, stratify=y
)

# Further split train into train + validation
val_ratio = 0.2
val_size = int(len(X_train_val) * val_ratio)
train_size = len(X_train_val) - val_size

X_train, X_val = X_train_val[:train_size], X_train_val[train_size:]
y_train, y_val = y_train_val[:train_size], y_train_val[train_size:]

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Dataset and DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define improved DBN-like model
class DBNNetImproved(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DBNNetImproved, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(0.4)

        self.fc2 = nn.Linear(512, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.3)

        self.out = nn.Linear(64, output_dim)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        return self.out(x)

input_dim = X_train_tensor.shape[1]
output_dim = len(le.classes_)

model = DBNNetImproved(input_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Early stopping parameters
patience = 15
best_val_loss = np.inf
epochs_no_improve = 0
n_epochs = 200

# Training loop with validation and early stopping
for epoch in range(n_epochs):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
    train_loss /= len(train_loader.dataset)

    # Validation step
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            outputs = model(xb)
            loss = criterion(outputs, yb)
            val_loss += loss.item() * xb.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{n_epochs} — Train Loss: {train_loss:.4f} — Val Loss: {val_loss:.4f}")

    # Check early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_model.pth")  # Save best model
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"Early stopping after {epoch+1} epochs")
            break

# Load best model
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

# Final evaluation on test data
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)

acc = accuracy_score(y_test_tensor.cpu(), predicted.cpu())
print(f"\n✅ Test Accuracy: {acc:.4f}")
print("\nClassification Report:\n", classification_report(y_test_tensor.cpu(), predicted.cpu()))


Epoch 1/200 — Train Loss: 0.7230 — Val Loss: 0.6959
Epoch 2/200 — Train Loss: 0.7027 — Val Loss: 0.6964
Epoch 3/200 — Train Loss: 0.6969 — Val Loss: 0.6976
Epoch 4/200 — Train Loss: 0.6952 — Val Loss: 0.6954
Epoch 5/200 — Train Loss: 0.6924 — Val Loss: 0.6958
Epoch 6/200 — Train Loss: 0.6934 — Val Loss: 0.6950
Epoch 7/200 — Train Loss: 0.6908 — Val Loss: 0.6972
Epoch 8/200 — Train Loss: 0.6896 — Val Loss: 0.6974
Epoch 9/200 — Train Loss: 0.6900 — Val Loss: 0.6973
Epoch 10/200 — Train Loss: 0.6899 — Val Loss: 0.6970
Epoch 11/200 — Train Loss: 0.6882 — Val Loss: 0.6973
Epoch 12/200 — Train Loss: 0.6842 — Val Loss: 0.7004
Epoch 13/200 — Train Loss: 0.6867 — Val Loss: 0.6993
Epoch 14/200 — Train Loss: 0.6868 — Val Loss: 0.7001
Epoch 15/200 — Train Loss: 0.6848 — Val Loss: 0.7019
Epoch 16/200 — Train Loss: 0.6858 — Val Loss: 0.6989
Epoch 17/200 — Train Loss: 0.6857 — Val Loss: 0.7021
Epoch 18/200 — Train Loss: 0.6824 — Val Loss: 0.7026
Epoch 19/200 — Train Loss: 0.6822 — Val Loss: 0.7036
Ep

In [ ]:
print(df["diagnosis"].value_counts()) # checking for class imbalance


diagnosis
0    7532
1    7468
Name: count, dtype: int64
